<a href="https://colab.research.google.com/github/mnansary/pyF2O/blob/master/colab_gen_unet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# colab specific task
*   mount google drive
*   TPU check
*   Check TF version

In [0]:
!pip3 install tensorflow==1.13.1

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

In [0]:

# tpu check
import os
import pprint
import tensorflow as tf
if 'COLAB_TPU_ADDR' not in os.environ:
  print('ERROR: Not connected to a TPU runtime; please see the first cell in this notebook for instructions!')
else:
  TPU_ADDRESS = 'grpc://' + os.environ['COLAB_TPU_ADDR']
  print ('TPU address is', TPU_ADDRESS)

  with tf.Session(TPU_ADDRESS) as session:
    devices = session.list_devices()
    
  print('TPU devices:')
  pprint.pprint(devices)



# ConvNet3D Model Training

#### Data 
* Set **FLAGS** and **PARAMS**
* Get **Train** and **Eval** Data


In [0]:
cd /content/gdrive/My\ Drive/PROJECTS/HACT/pyACTRECOG/

In [0]:
import sys
sys.path.append('.')

from coreLib.utils import readh5
import numpy as np 
class PARAMS:
    DS_DIR  = '/content/gdrive/My Drive/PROJECTS/HACT/Data/' # @param
    BATCH_SIZE      = 128  # @param
    NUM_EPOCHS      = 100  # @param
    IMAGE_DIM       = 32  # @param
    NB_CHANNELS     = 1    # @param
    X_TRAIN_IDEN    = 'X_Train.h5'  # @param
    Y_TRAIN_IDEN    = 'Y_Train.h5'  # @param
    X_EVAL_IDEN     = 'X_Eval.h5'  # @param
    Y_EVAL_IDEN     = 'Y_Eval.h5'  # @param
    MODEL_DIR       = '/content/gdrive/My Drive/PROJECTS/HACT/Model/' # @param
    MODEL_NAME      = 'convNet3D' # @param
    MIN_SEQ_LEN     = 6           #@param
    NB_CLASSES      = 17 #@param
X_train=readh5(os.path.join(PARAMS.DS_DIR,PARAMS.X_TRAIN_IDEN))
Y_train=readh5(os.path.join(PARAMS.DS_DIR,PARAMS.Y_TRAIN_IDEN))
X_eval=readh5(os.path.join(PARAMS.DS_DIR,PARAMS.X_EVAL_IDEN))
Y_eval=readh5(os.path.join(PARAMS.DS_DIR,PARAMS.Y_EVAL_IDEN))

X_train=X_train.astype('float32')/255.0
X_eval=X_eval.astype('float32')/255.0

print(X_train.shape)
print(Y_train.shape)
print(X_eval.shape)
print(Y_eval.shape)



#### COMPILE MODEL




In [0]:
import sys
sys.path.append('.')
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.optimizers import Adam
from coreLib.model import convNet3D

model=convNet3D(seq_len=PARAMS.MIN_SEQ_LEN,
                img_dim=PARAMS.IMAGE_DIM,
                nb_channels=PARAMS.NB_CHANNELS,
                nb_classes=PARAMS.NB_CLASSES)
model.summary()
model.compile(optimizer=Adam(), 
              loss=categorical_crossentropy,
              metrics=['accuracy'])



#### EXPORT TO TPU MODEL

In [0]:
TPU_WORKER = 'grpc://' + os.environ['COLAB_TPU_ADDR']
tf.logging.set_verbosity(tf.logging.INFO)
cluster_resolver=tf.contrib.cluster_resolver.TPUClusterResolver(TPU_WORKER)
strategy=tf.contrib.tpu.TPUDistributionStrategy(cluster_resolver)
model = tf.contrib.tpu.keras_to_tpu_model(model,strategy=strategy)


#### Train


In [0]:
history=model.fit(X_train,Y_train,
                  validation_data=(X_eval,Y_eval),
                  epochs=PARAMS.NUM_EPOCHS,
                  batch_size=PARAMS.BATCH_SIZE, 
                  verbose=1)


#### Save Model



In [0]:
model.save_weights(os.path.join(PARAMS.MODEL_DIR,'{}_final.h5'.format(PARAMS.MODEL_NAME)))

#### Plot Training Histoty

In [0]:
import matplotlib.pyplot as plt
%matplotlib inline
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('LOSS History')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.savefig(os.path.join(PARAMS.MODEL_DIR,'{}_history.png'.format(PARAMS.MODEL_NAME)))